In [1]:
import pandas as pd
import numpy as np
from timeit import timeit
from datetime import date, timedelta, datetime
import time

from FileLoad import Final_Load
from Skills import complex_skills

today = date.today()
tomorrow = (today + timedelta(days = 1)).strftime("%Y-%m-%d")

In [2]:
### Get data and mutate
df0 = Final_Load()

name_sort = {'Escalated':0, 'Unscheduled':1,'PNP Released':2,'Past Due':3,'Scheduled':4}
rm_sort = {'EMR Remote': 0, 'HIH - Other': 1, 'Onsite':2,'Offsite':3}
age_sort = {21: 0, 0: 1, 20:2, 15:3, 10:4, 5:5}

df0['name_sort'] = df0['Outreach Status'].map(name_sort)
df0['rm_sort'] = df0['Retrieval Group'].map(rm_sort)
df0['age_sort'] = df0['Group Number'].map(age_sort)

# df0['ORG_list'] = 0
df0['ORG Stat'] = 0 

df_dummy_status = pd.get_dummies(df0['Outreach Status'])
df_dummy_age = pd.get_dummies(df0['Group Number'])
df1 = df0.join(df_dummy_status).join(df_dummy_age)

### Unique Numbers count and status
df2 = df1.groupby(['PhoneNumber']).agg({'PhoneNumber':'count',21:'sum', 0:'sum', 20:'sum', 15:'sum', 10:'sum', 5:'sum','Cluster':'mean','Escalated':'sum','PNP Released':'sum','Past Due':'sum','Scheduled':'sum','Unscheduled':'sum',}).rename(columns={'PhoneNumber':'OutreachID Count','Cluster':'Cluster_Avg'})
df2 = df2.reset_index()

### Add info to main line and reskill
df3 = pd.merge(df0,df2, on='PhoneNumber')
df3 = complex_skills(df3)

### Sort Order and drop Dups
df4 = df3.sort_values(by = ['rm_sort', 'name_sort','age_sort', 'Unscheduled', 'Cluster_Avg'], ascending= [True,True, True, False, True]).reset_index(drop = True)

df4 = df4.drop_duplicates(['PhoneNumber']).reset_index(drop = True)
df4['ORG Stat'] = 1

Skipping line 56248: field larger than field limit (1000)
Skipping line 95215: field larger than field limit (1000)
Skipping line 96095: field larger than field limit (1000)
Skipping line 299151: field larger than field limit (1000)
Skipping line 300433: field larger than field limit (1000)
Skipping line 302367: field larger than field limit (1000)


In [4]:
df0['OutreachID'] = df0['OutreachID'].astype(str)


In [8]:
df0['OutreachID'] = df0['OutreachID'].astype(str)

df0['ORG list'] = df0.groupby(['PhoneNumber'])['OutreachID'].transform(lambda x : '|'.join(x))


28.3 s ± 723 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
df0['ORG list']

0                                         19863200|25618810
1                                19876666|24062010|25843036
2                       19881386|24023218|25828666|26248628
3                                         19881768|24028460
4         19882776|20461047|21605771|23233945|23770270|2...
                                ...                        
305370                                    25691517|27022307
305371                                             27022312
305372                                    26409642|27022319
305373                                             27022328
305374                                             27022329
Name: ORG list, Length: 294283, dtype: object

In [94]:
### Concat Column for ORGs to Unique Numbers
Phone_List = df4[df4['OutreachID Count'] > 1]['PhoneNumber']
df_clean = df0[['OutreachID', 'PhoneNumber']]
df_phone = pd.DataFrame()
new_row = pd.DataFrame()


In [95]:
def phone(number):
    df1 = df_clean[df_clean['PhoneNumber'] == number]['OutreachID'].tolist()
    string_ORG = ""
    for i in df1:
        string_ORG = string_ORG +str(i) +'|'
    new_row = [[number, string_ORG]]
    return new_row

In [97]:
# phone(8008604230)

In [89]:
startTime_1 = time.time()

for i in Phone_List:
    df_phone = df_phone.append(phone(i), ignore_index=True)

executionTime_1 = (time.time() - startTime_1)
print("-----------------------------------------------")
print('Time: ' + str(executionTime_1))
print("-----------------------------------------------")

-----------------------------------------------
Time: 355.47213912010193
-----------------------------------------------


In [93]:
df_phone

""


In [44]:
# df_phone['Cat ORGs'] = df_phone['Cat ORGs'].str.rstrip('dtype: object')#.str[3:]
%timeit
df_phone['Cat ORGs'] = df_phone['Cat ORGs']


,PhoneNumber,Cat ORGs
0,9.375231e+09,0 22093445|23241175|23241895|23242715|23812...
1,4.192273e+09,0 23836901|24321202|24985548|25354434|25555...
2,2.709655e+09,0 24313824|25318568|25406491|25600728|26035...
3,4.195178e+09,0 22299060|25263160| dtype: object
4,8.647203e+09,0 24763773|25396363|25979483|25979836|25979...
...,...,...
59348,4.053309e+09,0 26470629|26473861| dtype: object
59349,4.056924e+09,0 26471092|26471887| dtype: object
59350,6.418121e+09,0 26472470|26472550| dtype: object
59351,9.128197e+09,0 26474226|26475347|26475351| dtype: object


In [37]:
len('| dtype: object')
%timeit

In [ ]:
os.getcwd()

In [ ]:
# df_phone.to_csv('df_phone.csv', index=False)
